# Notebook de test des données lapi post-traitees

In [45]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
import donnees_posttraitees as t
import graphs as g
import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os, math
from sklearn.cluster import DBSCAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. import des données, traitement de mise en forme
## 1.1 Import
Les données sont importées sur la période du 24/01/19 au 23/02/19, sur laquelle il n'y a pas de manque de données à priori

In [ ]:
df_passages, df_plaques, df_immat=t.ouvrir_fichier_lapi_final('2019-01-24 00:00:00','2019-02-13 23:59:59')

## 1.2. Suppression des doublons

In [9]:
df_passages=t.supprimer_doublons(df_passages)

## 1.3 Ajout de l'attribut de différenciation VL / PL

In [39]:
%time df_passages2=t.affecter_type(df_passages,df_immat)

Wall time: 3min 14s


***ATTENTION : ENCORE 18764 IMMATRCULATION COURTE PRESENTES DANS TE_PASSAGES ET NON PRENSENTES DANS TE_PLAQUES_COURTE :***

In [ ]:
df_passages2.loc[df_passages2.immatriculation.isna()]

***ATTENTION : PLUS DE 1M D'IMMATRCULATION SANS VALEUR DE VL_TOT ou PL_TOT ou VUL_TOT :***

In [19]:
df_passages2.loc[df_passages2['type_veh']==-1]

In [ ]:
df_immat.loc[df_immat['immatriculation']=='000013c5f1fc019be0ac5a49a32f675be089542a']

# 2. Identifier les trajets de transit

In [42]:
dico_od,  dico_passag, dico_tps_max=t.transit_temps_complet('2019-01-29 00:00:00',3,df_passages2)

date : 2019-01-29 00:00:00 debut_traitement : 2019-05-11 23:23:32.817432
date : 2019-01-30 00:00:00 debut_traitement : 2019-05-11 23:25:04.342900
date : 2019-01-31 00:00:00 debut_traitement : 2019-05-11 23:26:34.701772


# 3. Filtrer les trajets selon un temps de parcours

In [43]:
#df des temsp de parcours
dixco_tpsmax_corrige=t.corriger_df_tps_parcours(dico_tps_max)

***Ici on pourra caracteriser l'impact de la marge***

In [44]:
# df complete avec l'atribut filtre
df_transit_avec_filtre=t.jointure_temps_reel_theorique(dico_od,dixco_tpsmax_corrige,t.liste_complete_trajet, 15)

In [47]:
#filtrer le df : 
dico_od_final=df_transit_avec_filtre.loc[df_transit_avec_filtre['filtre_tps']==1]

## Correction des trajets depuis ou vers A660 relatifs à A63

In [48]:
%time df_od_corrige=t.correction_trajet(df_passages2, dico_od_final)

Wall time: 7min 6s


# extrapolation des trajets non complets

In [50]:
#creer les donnees necessaires
%time df_filtre_A63,df_passage_transit,df_non_transit=t.param_trajet_incomplet('2019-01-28 00:00:00',df_od_corrige,df_passages2,dico_passag)

Wall time: 1min 45s


In [63]:
#trajet de transit incomplet sans filtre sur temps de parcours
%time trajet_transit_incomplet2, passage2=t.transit_trajet_incomplet(df_filtre_A63,df_passage_transit,'2019-01-29 00:00:00',1, df_passages2)

date : 2019-01-29 00:00:00 debut_traitement : 2019-05-12 00:17:32.032862
Wall time: 11.5 s


In [ ]:
#filtrer selon le temps de parcours
trajet_transit_incomplet_filtre_temps=t.jointure_temps_reel_theorique(trajet_transit_incomplet2,dixco_tpsmax_corrige,t.liste_trajet_incomplet, 15,'incomplet')
#mettre en forme les attributs
trajet_transit_incomplet_filtre_temps=trajet_transit_incomplet_filtre_temps[['date_cam_1', 'immat', 'state', 'l', 'date_cam_2', 'cameras', 'origine',
       'destination', 'o_d', 'tps_parcours', 'period', 'date', 'temps', 'type',
       'tps_parcours_theoriq', 'filtre_tps']]
#filtrer le df : 
trajet_transit_incomplet_final=trajet_transit_incomplet_filtre_temps.loc[trajet_transit_incomplet_filtre_temps['filtre_tps']==1]
#ajouter l'attribut d'identification des trajets 
trajet_transit_incomplet_final['correction_o_d']=True
#6. Ajouter au df des o_d précédents : 
df_od_final=pd.concat([df_od_corrige,trajet_transit_incomplet_final])

# 4. CREER LE TABLEAU DE MATRICE O-D

In [65]:
pd.pivot_table(dico_od_final,values='l', index='origine', columns='destination',aggfunc='count', margins=True).fillna('NC')

destination,A10,A62,A63,A660,A89,N10,All
origine,,,,,,,
A10,NC,919,1316,306,45,NC,2586
A62,451,NC,74,169,93,579,1366
A63,749,40,NC,NC,217,4431,5437
A660,598,160,NC,NC,277,2631,3666
A89,47,160,447,219,NC,193,1066
N10,NC,604,4389,802,104,NC,5899
All,1845,1883,6226,1496,736,7834,20020
